# **Chloroplast Curation**

### Simón Villanueva

El proposito de este notebook es describir claramente los problemas que se tienen con los CDS en la anotación del cloroplasto de Sacha Inchi.

El enfoque aquí son los CDS ya que los tRNAs han sido más sujetos de revisión anteriormente y los rRNAs parecen no tener problemas.

Además se revisa la anotación de cloroplasto chino.

***

Primero cargamos el genbank actual que tenemos de Sacha

In [1]:
from Bio import SeqIO

In [2]:
sacha = SeqIO.read("../../../Data/Processed/Chloroplast_annotations/Corrected_annotation_wo_ref/Sacha_denovo_tscan_woref/Sacha_denovo_Tscan_wref_Plukenetia-chloroplast_GenBank.gb","gb")

Ahora, ¿qué características deben tener los genes codificantes? Bueno, que sean fácilmente chequeables, vienen a la mente 3:

- Deben iniciar por un codón de inicio (Metionina).
- Deben contener uno y sólo un codón de parada, el cuál debe ubicarse al final del último codón.
- Debe estar exactamente dividio en codones (sin que queden nucleotidos sobrando).

Entonces, haremos una pequeña función que chequee estas condiciones y reporte en una tabla aquellos CDS que sean anómalos.

In [20]:
def check_start(cds, gen_cod):
    "Devuelve True si la protina empieza por metionina si no False"
    return True if cds.seq.translate(table = gen_cod)[0] == "M" else False

def check_exact_cod(cds):
    "Cálcula cuantos nucleotidos le 'sobran' a la proteina"
    return len(cds.seq) % 3

def check_stops(cds, gen_cod):
    """Devuelve el número de codones de parada si hay más de uno
    Si hay exactamente uno, devuelve True si está en la última 
    posición o si no False"""
    
    prot = cds.seq.translate(table = gen_cod)
    stops = prot.count("*")
    if stops != 1:
        return stops
    else:
        return True if prot[-1] == "*" else False

def make_table(bad_cds):
    print("index\tcds\tstart\tcod_excess\tstop")
    for cds, reasons in bad_cds.items():
        aux_list = [str(reasons["index"]),cds]
        aux_list.append(str(reasons["start"]))
        aux_list.append(str(reasons["cod"]))
        aux_list.append(str(reasons["end"]))
        print("\t".join(aux_list))
    
def check_cds(req, gen_cod = 11):
    anomalus = {}
    for i,ft in enumerate(req.features):
        if ft.type == "CDS":
            reasons = {}
            cds = ft.extract(req)
            reasons["start"] = check_start(cds, gen_cod)
            reasons["cod"] = check_exact_cod(cds)
            reasons["end"] = check_stops(cds, gen_cod)
            
            if reasons["start"] != True or reasons["cod"] != 0 or reasons["end"] != True:
                reasons["index"] = i
                anomalus[ft.qualifiers["gene"][0]] = reasons
    make_table(anomalus)
    

In [21]:
check_cds(sacha)

index	cds	start	cod_excess	stop
50	psbC	False	0	True
71	rpoC1	True	1	0
83	atpI	False	0	True
87	atpF	True	2	13
110	ndhK	True	2	0
187	petB	False	1	13
190	petD	False	2	18
200	infA	False	0	2
206	rpl16	False	1	10
212	rpl22	True	1	5
370	rps19	False	0	True
274	ycf1	True	1	False
288	ndhI	True	0	0
291	ndhG	True	0	2
297	ndhD	False	0	True
301	ccsA	True	2	0
309	ndhF	True	1	0
315	ycf1-fragment	True	2	0


Así, parece que tenemos 18 CDS listados, de los cuales deberiamos revisar al menos 17 (ycf1-fragment está bien que sea así). Entonces tenemos:

- Proteinas que no inician en Metionina: 8
- Proteinas con exceso de nucleotidos: 10
- Proteinas con Stops anormales: 13

Pueden ser variadas las causales de esto, pero lo más probable es que al anotador le haya faltado un nucleotido que haya cambiado el marco de lectura; o que afecte precisamente el codón de inicio o de parada. Estas son el tipo de cosas que deberiamos revisar en primera instancia.